In [21]:
# linear algebra
import numpy as np
# data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
#Unix commands
import os

# import useful tools
from glob import glob
from PIL import Image
#import cv2

# import data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs

# import data augmentation
import albumentations as albu

# import math module
import math

In [25]:
#Libraries
#import pandas_profiling
import pandas as pd
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

In [26]:
# One-hot encoding
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Setup the paths to train and test images
DATASET = r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression'
TEST_DIR = r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\test'
TRAIN_CSV_PATH = r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\train.csv'

# Glob the directories and get the lists of train and test images
train_fns = glob(DATASET + '*')
test_fns = glob(TEST_DIR + '*')

In [28]:
# Loading training data and test data
train = pd.read_csv(r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\train.csv')
test = pd.read_csv(r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\test.csv')
test.head()

AttributeError: 'NoneType' object has no attribute 'items'

                     Patient  Weeks   FVC    Percent  Age   Sex SmokingStatus
0  ID00419637202311204720264      6  3020  70.186855   73  Male     Ex-smoker
1  ID00421637202311550012437     15  2739  82.045291   68  Male     Ex-smoker
2  ID00422637202311677017371      6  1930  76.672493   73  Male     Ex-smoker
3  ID00423637202312137826377     17  3294  79.258903   72  Male     Ex-smoker
4  ID00426637202313170790466      0  2925  71.824968   73  Male  Never smoked

In [14]:
#Loading Sample Files for Submission
sample = pd.read_csv(r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\sample_submission.csv')
# Confirmation of the format of samples for submission
sample.head(3)

AttributeError: 'NoneType' object has no attribute 'items'

                    Patient_Week   FVC  Confidence
0  ID00419637202311204720264_-12  2000         100
1  ID00421637202311550012437_-12  2000         100
2  ID00422637202311677017371_-12  2000         100

In [29]:
#Loading Sample Files for Submission
sample = pd.read_csv(r'C:\Users\akhter\Documents\ds project\osic_pulmonary_fibrosis_progression\sample_submission.csv')

In [30]:
# display the smoking status of the training data without duplicates
print(train['SmokingStatus'].drop_duplicates())

0             Ex-smoker
36         Never smoked
288    Currently smokes
Name: SmokingStatus, dtype: object


In [17]:
# display the training data without gender duplication
print(train['Sex'].drop_duplicates())

0       Male
62    Female
Name: Sex, dtype: object


In [18]:
# Display some of the training data
train.head(10)

AttributeError: 'NoneType' object has no attribute 'items'

                     Patient  Weeks   FVC    Percent  Age   Sex SmokingStatus
0  ID00007637202177411956430     -4  2315  58.253649   79  Male     Ex-smoker
1  ID00007637202177411956430      5  2214  55.712129   79  Male     Ex-smoker
2  ID00007637202177411956430      7  2061  51.862104   79  Male     Ex-smoker
3  ID00007637202177411956430      9  2144  53.950679   79  Male     Ex-smoker
4  ID00007637202177411956430     11  2069  52.063412   79  Male     Ex-smoker
5  ID00007637202177411956430     17  2101  52.868646   79  Male     Ex-smoker
6  ID00007637202177411956430     29  2000  50.327126   79  Male     Ex-smoker
7  ID00007637202177411956430     41  2064  51.937594   79  Male     Ex-smoker
8  ID00007637202177411956430     57  2057  51.761449   79  Male     Ex-smoker
9  ID00009637202177434476278      8  3660  85.282878   69  Male     Ex-smoker

In [ ]:
# Check for missing values in the training data
train.isnull().sum()

In [ ]:
# Let's check the max value and the max value for Weeks
print("Minimum number of value for Weeks is: {}".format(train['Weeks'].min()), "\n" +
      "Maximum number of value for Weeks is: {}".format(train['Weeks'].max() ))

In [ ]:
# Check the Patient statistics of the training data
train['Patient'].describe()

In [ ]:
# Check the FVC statistics of the training data
train['FVC'].describe(percentiles=[0.1,0.2,0.5,0.75,0.9])

In [ ]:
# Check age-related statistics in the training data
train['Age'].describe()

In [ ]:
# Display of test data
print(test)

In [ ]:
# Combine the Patient ID and Week columns
test_patient_weeklist = test['Patient_Week'] = test['Patient'].astype(str)+"_"+test['Weeks'].astype(str)
test2 = test.drop('Patient', axis=1)
test3 = test.drop('Weeks', axis=1)
test4 = test.reindex(columns=['Patient_Week', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus'])
test4.head(7)

In [ ]:
# Find the unique number of patient IDs. 
n = train['Patient'].nunique()
print(n)

In [ ]:
# First, I'll use Sturgess's formula to find the appropriate number of classes in the histogram 
k = 1 + math.log2(n)

In [ ]:
# Display a histogram of the FVC of the training data
sns.distplot(train['FVC'], kde=True, rug=False, bins=int(k)) 
# Graph Title
plt.title('FVC')
# Show Histogram
plt.show()

In [ ]:
# Display a histogram of the age of the training data
sns.distplot(train['Age'], kde=True, rug=False, bins=int(k)) 
# Title of the study data age graph
plt.title('Age')
# Display a histogram of the age of the training data
plt.show()

In [ ]:
# Show the correlation between age and FVC in the training data
sns.scatterplot(data=train, x='Age', y='FVC')

In [ ]:
# Produce correlation coefficients between age and FVC of the training data
df = train
df.corr()['Age']['FVC']

In [ ]:
# Narrowing down to smokers in the training data to produce a correlation coefficient between age and FVC 
df_smk = train.query('SmokingStatus == "Currently smokes"')

df_smk.corr()['Age']['FVC']

In [ ]:
# Scatterplots of age and FVC for training data extracted by smokers
sns.scatterplot(data=df_smk, x='Age', y='FVC')

In [ ]:
# Show the correlation between age and FVC in the training data
sns.scatterplot(data=train, x='Percent', y='FVC')

In [ ]:
# Compute summary statistics for FVC aggregated by age
df.groupby('Age').describe()['FVC']

In [ ]:
# Calculate summary statistics for FVC aggregated by patient ID 
df.groupby('Patient').describe(percentiles=[0.1,0.2,0.5,0.8])['FVC']

In [ ]:
df_corr = df.corr()
print(df_corr)

In [ ]:
# View the correlation heat map
corr_mat = df.corr(method='pearson')
sns.heatmap(corr_mat,
            vmin=-1.0,
            vmax=1.0,
            center=0,
            annot=True, # True:Displays values in a grid
            fmt='.1f',
            xticklabels=corr_mat.columns.values,
            yticklabels=corr_mat.columns.values
           )
plt.show()

In [ ]:
# Draw a pie chart about gender.
plt.pie(train["Sex"].value_counts(),labels=["Male","Female"],autopct="%.1f%%")
plt.title("Ratio of Sex")
plt.show()

In [ ]:
# Draw a pie chart about smoking status
plt.pie(train["SmokingStatus"].value_counts(),labels=["Ex-smoker","Never smoked","Currently smokes"],autopct="%.1f%%")
plt.title("SmokingStatus")
plt.show()

In [ ]:
print(train[train.FVC < 1651])

In [ ]:
# Image loading
#../input/osic-pulmonary-fibrosis-progression/
im_bad1 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00023637202179104603099/3.dcm")
im_bad2 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00023637202179104603099/5.dcm")
im_bad3 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00023637202179104603099/7.dcm")
im_bad4 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00023637202179104603099/15.dcm")

# Converting an image to an array
im_list1 = np.asarray(im_bad1)
im_list2 = np.asarray(im_bad2)
im_list3 = np.asarray(im_bad3)
im_list4 = np.asarray(im_bad4)

# I'll paste the image.
plt.imshow(im_list1)
# View image and paste the next image
plt.title("71, Female")
plt.show()
plt.imshow(im_list2)
# View image and paste the next image
plt.title("71, Female")
plt.show()
plt.imshow(im_list3)
# View image and paste the next image
plt.title("71, Female")
plt.show()
plt.imshow(im_list4)

# View image
plt.title("71, Female")
plt.show()

In [ ]:
print(train[train.FVC > 3874])

In [ ]:
# Image loading
im_good1 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00009637202177434476278/11.jpg")
im_good2 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00014637202177757139317/2.jpg")
im_good3 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00032637202181710233084/30.jpg")
im_good4 = Image.open("../input/osic-pulmonary-jpg/processed_images/train/ID00032637202181710233084/35.jpg")

# Converting an image to an array
im_list10 = np.asarray(im_good1)
im_list11 = np.asarray(im_good2)
im_list12 = np.asarray(im_good3)
im_list13 = np.asarray(im_good4)

# I'll paste the image.
plt.imshow(im_list10)
# View image and paste the next image
plt.title("69, Male")
plt.show()
plt.imshow(im_list11)
# View image and paste the next image
plt.title("56, Male")
plt.show()
plt.imshow(im_list12)
# View image and paste the next image
plt.title("63, Male")
plt.show()
plt.imshow(im_list13)

# View image
plt.title("63, Male")
plt.show()

In [ ]:
# Create models and train them with training data
train_x = train.drop(['FVC'], axis=1)
train_y = df['FVC']

In [ ]:
# Confirmation of current value
print(train_x)

In [ ]:
train_x['Patient_Week'] = train_x['Patient'].astype(str)+"_"+train_x['Weeks'].astype(str)
train_x.head(5)

In [ ]:
#Conversion of category variables to arbitrary values
train_x['Sex'] = train_x['Sex'].map({'Male': 0, 'Female': 1})
train_x['SmokingStatus'] = train_x['SmokingStatus'].map({'Never smoked': 0, 'Ex-smoker': 1, 'Currently smokes': 2})

In [ ]:
# Combine the Patient ID and Week columns
train_x_patient_weeklist = train_x['Patient_Week'] = train_x['Patient'].astype(str)+"_"+train_x['Weeks'].astype(str)
train_x2 = train_x.drop('Patient', axis=1)
train_x3 = train_x.drop('Weeks', axis=1)
train_x4 = train_x.reindex(columns=['Patient_Week', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus'])
train_x4.head(7)
# Confirming the converted value
print(train_x4)

In [ ]:
# The test data is only features, so I'll just copy it
test_x = test.copy()

In [ ]:
osic_features = ['Percent', 'Age', 'Sex', 'SmokingStatus']

In [ ]:
# Define model. Specify a number for random_state to ensure same results each run
osic_model = DecisionTreeRegressor(random_state=1)

# Fit model
osic_model.fit(X, train_y)

In [ ]:
print(X.head())
print("The predictions are")
print(osic_model.predict(X.head()))

In [ ]:
# Let's visualize the FVC of Training Data
plt.figure(figsize=(18,6))
plt.plot(train_x4["FVC"], label = "Train_Data")
plt.legend()
plt.show()

In [ ]:
# Let's visualize the FVC predictions
plt.figure(figsize=(18,6))

Y_train_Graph = pd.DataFrame(X)
plt.plot(Y_train_Graph, label = "Predict")
plt.legend()

In [ ]:
#Reading the file
submission = pd.DataFrame(columns = ["Patient_Week", "FVC", "Confidence"])
#Exporting Files
submission.to_csv('submission.csv', index=False)
print()